In [1]:
import scipy
import pandas as pd
import numpy as np
import math
import pymongo
import MySQLdb as sql
import _mysql
import random
import csv
import time
import re
import matplotlib.pyplot as plt; import matplotlib.pylab as pylab
#%matplotlib inline
pd.options.display.mpl_style = 'default'
pylab.rcParams['figure.figsize'] = 12, 6
from dateutil import parser
import Quandl
from pymongo import MongoClient
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import urllib2

## Get A BeautifulSoup Object

In [121]:
class BloombergSearch:
    def __init__(self, search_term):
        self.search_term = search_term
        self.url_page1 = ('http://www.bloomberg.com/search?query=' + str(self.search_term))

    def get_search_soup(self):
        url =  self.url_page1
        soup = self.get_soup(url)
        return soup
    def get_soup(self, url):
        page = urllib2.urlopen(url).read()
        soup = BeautifulSoup(page)
        return soup
    def get_search_page_links(self, num_pages):
        article_list = []
        for i in range(1, num_pages + 1):
            temp_soup = self.get_soup(self.url_page1 + str('&page=') + str(i))
            for result in temp_soup.find_all('h1'):
                try:
                    if 'video' in result.a['href']:
                        continue
                    if 'http' in result.a['href']:
                        #print item.a['href']
                        article_list.append(result.a['href'])
                    else:
                        #print 'http://www.bloomberg.com/' + item.a['href']
                        article_list.append('http://www.bloomberg.com/' + result.a['href'])
                except:
                    continue
            #print 'Added page=' + str(i)
        return article_list
    
    def get_post_body(self, article_url):
        final_text = ""
        article_soup = self.get_soup(article_url)
        query = article_soup.find_all('div',  class_="article-body__content")
        for item in query:
            for text in item.find_all('p'):
                final_text = final_text + '\n\n' + str(text.text.encode('utf-8'))
        if final_text == "":
            return 0
        return final_text
    
    def get_post_date(self, article_url):
        final_text = ""
        article_soup = self.get_soup(article_url)
        result = article_soup.find('time', class_ = "published-at")
        try:
            return result['datetime']
        except:
            return None
    
    def get_post_author(self, article_url):
        final_text = ""
        article_soup = self.get_soup(article_url)
        result = article_soup.find('a', class_ = "author-link")
        try:
            return result.text.lstrip().rstrip()
        except:
            return None
    
    def get_post_title(self, article_url):
        final_text = ""
        article_soup = self.get_soup(article_url)        
        result = article_soup.find('title')
        try:
            return result.text.lstrip().rstrip()
        except:
            return None
    
    def make_info(self, pages = 1):
        final_df = pd.DataFrame()
        for url in self.get_search_page_links(pages):
            body = self.get_post_body(url)
            title = self.get_post_title(url)
            author = self.get_post_author(url)
            date = self.get_post_date(url)
            temp_series = pd.Series([title, author, date, body])
            final_df= final_df.append(temp_series, ignore_index = True)
        final_df.columns = ['title', 'author', 'date', 'text']
        return final_df

In [122]:
bloom_obj = BloombergSearch('apple')

In [123]:
temp = bloom_obj.make_info(3)
print len(url_list)
temp

32


,title,author,date,text
0,Apple-FBI Row Spurs Senate Bill on Encrypted D...,None,2016-02-23T19:10:03.519Z,\n\nApple Inc. and other U.S. companies wouldn...
1,Bloomberg Business,None,2016-02-23T16:01:14.766Z,\n\nBill Gates vigorously denies a report that...
2,Bill Gates ‘Disappointed’ by Reports He Backs ...,Jing Cao,2016-02-23T15:09:38.332Z,\n\nBill Gates is “disappointed” with a recent...
3,Apple v. FBI? Let Congress Decide - Bloomberg ...,None,None,0
4,Apple Says It Received Nine Similar iPhone Sea...,Tiffany Kary,2016-02-23T14:43:11.774Z,\n\nProsecutors across the U.S. have asked App...
5,Bill Gates Sides With Government in Apple Clas...,Edwin Chan,2016-02-23T05:14:03.045Z,\n\nMicrosoft Corp. co-founder Bill Gates has ...
6,Samsung Opens New Phase in Apple Battle With G...,Jungah Lee,2016-02-22T22:47:46.391Z,\n\nSamsung Electronics Co. showed off virtual...
7,Apple Calls on Congress to Form Committee for ...,Ville Heiskanen,2016-02-22T12:09:16.869Z,\n\nApple Inc. said the government should with...
8,China's Huawei Backs Apple Stance in Phone Unl...,Caroline Hyde,2016-02-22T02:43:29.839Z,\n\nHuawei Technologies Co. will back Apple In...
9,Apple's New Lawyer Calls iPhone-Unlock Order a...,Miles Weiss,2016-02-21T20:33:27.689Z,"\n\nApple Inc.’s newly hired outside lawyer, i..."
